# Project Proposal (Matt)

ADD TEXT HERE

# Project Code

## Data Extraction

### Flight Data (Myles & Sedra)

MK NOTES
1. Flight Data - Columns to Drop: 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
2. Flight Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
3. Flight Data - Add Column for Total Delay (Add Together Departure & Arrival Delay Values)
4. Flight Data - Combine Date Value Columns into Single Date Column
5. Airline Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
6. Airport Data - Columns to Drop: 'LATITUDE', 'LONGITUDE'

In [1]:
#Import Modules
import os
import pandas as pd
import numpy as np
import datetime

In [2]:
flight_path = "Flight data/flights.csv"
airline_path = "Flight data/airlines.csv"
airport_path = "Flight data/airports.csv"

In [3]:
flights_pd = pd.read_csv(flight_path)
flights_pd.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,2,1,7,AA,2400,N3JKAA,LAX,DFW,5,...,452.0,-2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,2,1,7,AS,98,N794AS,ANC,SEA,5,...,501.0,32.0,0,0,NaN,3.0,0.0,29.0,0.0,0.0
2,2015,2,1,7,AA,258,N3FEAA,LAX,MIA,20,...,849.0,45.0,0,0,NaN,45.0,0.0,0.0,0.0,0.0
3,2015,2,1,7,DL,806,N962DN,SFO,MSP,20,...,548.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,2,1,7,NK,612,N604NK,LAS,MSP,25,...,515.0,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
airport_pd = pd.read_csv(airport_path)
airport_pd.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [5]:
airline_pd = pd.read_csv(airline_path)
airline_pd.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [6]:
update1_flights = flights_pd.drop(columns=['CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'])
update1_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED
0,2015,2,1,7,AA,2400,N3JKAA,LAX,DFW,5,...,170.0,150.0,1235,447.0,5.0,454,452.0,-2.0,0,0
1,2015,2,1,7,AS,98,N794AS,ANC,SEA,5,...,207.0,190.0,1448,454.0,7.0,429,501.0,32.0,0,0
2,2015,2,1,7,AA,258,N3FEAA,LAX,MIA,20,...,339.0,258.0,2342,836.0,13.0,804,849.0,45.0,0,0
3,2015,2,1,7,DL,806,N962DN,SFO,MSP,20,...,216.0,192.0,1589,538.0,10.0,600,548.0,-12.0,0,0
4,2015,2,1,7,NK,612,N604NK,LAS,MSP,25,...,179.0,164.0,1299,511.0,4.0,526,515.0,-11.0,0,0


In [7]:
update1_flights['Total Delay'] = update1_flights['SCHEDULED_DEPARTURE'] + update1_flights['ARRIVAL_DELAY']
update1_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,Total Delay
0,2015,2,1,7,AA,2400,N3JKAA,LAX,DFW,5,...,150.0,1235,447.0,5.0,454,452.0,-2.0,0,0,3.0
1,2015,2,1,7,AS,98,N794AS,ANC,SEA,5,...,190.0,1448,454.0,7.0,429,501.0,32.0,0,0,37.0
2,2015,2,1,7,AA,258,N3FEAA,LAX,MIA,20,...,258.0,2342,836.0,13.0,804,849.0,45.0,0,0,65.0
3,2015,2,1,7,DL,806,N962DN,SFO,MSP,20,...,192.0,1589,538.0,10.0,600,548.0,-12.0,0,0,8.0
4,2015,2,1,7,NK,612,N604NK,LAS,MSP,25,...,164.0,1299,511.0,4.0,526,515.0,-11.0,0,0,14.0


In [8]:
update3 = update1_flights[(update1_flights['DAY'] >= 16) & (update1_flights['DAY'] <= 24)]
update3.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,Total Delay
225468,2015,2,16,1,AA,2400,N5ESAA,LAX,DFW,5,...,147.0,1235,443.0,6.0,453,449.0,-4.0,0,0,1.0
225469,2015,2,16,1,DL,1745,N365NW,SMF,MSP,5,...,183.0,1517,511.0,3.0,529,514.0,-15.0,0,0,-10.0
225470,2015,2,16,1,DL,2579,N693DL,DEN,ATL,15,...,135.0,1199,524.0,11.0,511,535.0,24.0,0,0,39.0
225471,2015,2,16,1,US,2020,N917US,PHX,CLT,15,...,186.0,1773,531.0,10.0,600,541.0,-19.0,0,0,-4.0
225472,2015,2,16,1,AA,258,N3DEAA,LAX,MIA,20,...,254.0,2342,750.0,8.0,804,758.0,-6.0,0,0,14.0


In [9]:
update3['DATE'] = update3['YEAR'].astype(str) + '-0' + update3['MONTH'].astype(str) + '-' + update3['DAY'].astype(str)
update4 = update3.drop(['YEAR', 'MONTH', 'DAY'], axis = 1)
update4.head()

C:\Users\sedra\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,Total Delay,DATE
225468,1,AA,2400,N5ESAA,LAX,DFW,5,3.0,-2.0,13.0,...,1235,443.0,6.0,453,449.0,-4.0,0,0,1.0,2015-02-16
225469,1,DL,1745,N365NW,SMF,MSP,5,2359.0,-6.0,9.0,...,1517,511.0,3.0,529,514.0,-15.0,0,0,-10.0,2015-02-16
225470,1,DL,2579,N693DL,DEN,ATL,15,20.0,5.0,49.0,...,1199,524.0,11.0,511,535.0,24.0,0,0,39.0,2015-02-16
225471,1,US,2020,N917US,PHX,CLT,15,13.0,-2.0,12.0,...,1773,531.0,10.0,600,541.0,-19.0,0,0,-4.0,2015-02-16
225472,1,AA,258,N3DEAA,LAX,MIA,20,20.0,0.0,16.0,...,2342,750.0,8.0,804,758.0,-6.0,0,0,14.0,2015-02-16


In [10]:
#Flight Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
array = ['UA', 'AA', 'US', 'WN', 'DL', 'VX']
update5 = update4.loc[update4['AIRLINE'].isin(array)]
update5.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,Total Delay,DATE
225468,1,AA,2400,N5ESAA,LAX,DFW,5,3.0,-2.0,13.0,...,1235,443.0,6.0,453,449.0,-4.0,0,0,1.0,2015-02-16
225469,1,DL,1745,N365NW,SMF,MSP,5,2359.0,-6.0,9.0,...,1517,511.0,3.0,529,514.0,-15.0,0,0,-10.0,2015-02-16
225470,1,DL,2579,N693DL,DEN,ATL,15,20.0,5.0,49.0,...,1199,524.0,11.0,511,535.0,24.0,0,0,39.0,2015-02-16
225471,1,US,2020,N917US,PHX,CLT,15,13.0,-2.0,12.0,...,1773,531.0,10.0,600,541.0,-19.0,0,0,-4.0,2015-02-16
225472,1,AA,258,N3DEAA,LAX,MIA,20,20.0,0.0,16.0,...,2342,750.0,8.0,804,758.0,-6.0,0,0,14.0,2015-02-16


In [11]:
#calculations of early arrivals
earlyarrivals = update5[update5['Total Delay']<=0]

scheduled_arrival = []

for item in earlyarrivals['SCHEDULED_ARRIVAL'].astype(int):
    if item == 2400:
        scheduled_arrival.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        scheduled_arrival.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        scheduled_arrival.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
          scheduled_arrival.append(datetime.datetime.strptime('00' + str(item), '%H%M').strftime('%H:%M'))
    elif item <= 9:
        scheduled_arrival.append(datetime.datetime.strptime('000' + str(item), '%H%M').strftime('%H:%M'))
        
earlyarrivals['SCHEDULED_ARRIVAL'] = scheduled_arrival


#early (Scheduled) departures
earlydeparture = update5[update5['Total Delay']<=0]

scheduled_departure = []

for item in earlydeparture['SCHEDULED_DEPARTURE'].astype(int):
    if item == 2400:
        scheduled_departure.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        scheduled_departure.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        scheduled_departure.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        scheduled_departure.append(datetime.datetime.strptime('00' + str(item), '%H%M').strftime('%H:%M'))
    elif item <= 9:
        scheduled_departure.append(datetime.datetime.strptime('000' + str(item), '%H%M').strftime('%H:%M'))
        
earlydeparture['SCHEDULED_DEPARTURE'] = scheduled_departure


#early departure times
EARLYDEPARTURETIME = update5[update5['Total Delay']<=0]

departure_time = []

for item in earlydeparture['DEPARTURE_TIME'].astype(int):
    if item == 2400:
        departure_time.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        departure_time.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        departure_time.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
          departure_time.append(datetime.datetime.strptime('00' + str(item), '%H%M').strftime('%H:%M'))
    elif item <= 9:
        departure_time.append(datetime.datetime.strptime('000' + str(item), '%H%M').strftime('%H:%M'))
        
EARLYDEPARTURETIME['DEPARTURE_TIME'] = departure_time


#early arrival times
EARLYARRIVALTIME = update5[update5['Total Delay']<=0]

arrival_time = []

for item in EARLYARRIVALTIME['ARRIVAL_TIME'].astype(int):
    if item == 2400:
        arrival_time.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        arrival_time.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        arrival_time.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        arrival_time.append(datetime.datetime.strptime('00' + str(item), '%H%M').strftime('%H:%M'))
    elif item <= 9:
        arrival_time.append(datetime.datetime.strptime('000' + str(item), '%H%M').strftime('%H:%M'))
        
EARLYARRIVALTIME['ARRIVAL_TIME'] = arrival_time


#early wheels off
EARLYWHEELSOFF = update5[update5['Total Delay']<=0]

wheels_off = []

for item in EARLYWHEELSOFF['WHEELS_OFF'].astype(int):
    if item == 2400:
        wheels_off.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        wheels_off.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        wheels_off.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        wheels_off.append(datetime.datetime.strptime('00' + str(item), '%H%M').strftime('%H:%M'))
    elif item <= 9:
        wheels_off.append(datetime.datetime.strptime('000' + str(item), '%H%M').strftime('%H:%M'))
        
EARLYWHEELSOFF['WHEELS_OFF'] = wheels_off

#EARLY "WHEELS ON"
EARLYWHEELSON = update5[update5['Total Delay']<=0]

wheels_on = []

for item in EARLYWHEELSON['WHEELS_ON'].astype(int):
    if item == 2400:
        wheels_on.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        wheels_on.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        wheels_on.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        wheels_on.append(datetime.datetime.strptime('00' + str(item), '%H%M').strftime('%H:%M'))
    elif item <= 9:
        wheels_on.append(datetime.datetime.strptime('000' + str(item), '%H%M').strftime('%H:%M'))
        
EARLYWHEELSON['WHEELS_ON'] = wheels_on


#idk

departure_delay = []
arrival_delay = []
total_delay = []

for item in earlyarrivals['DEPARTURE_DELAY'].astype(int):
    if item < 0:
        departure_delay.append('-' + str(datetime.timedelta(minutes = (-1 * item)))[:-3])
    else:
        departure_delay.append(str(datetime.timedelta(minutes = item))[:-3])
        
for item in earlyarrivals['ARRIVAL_DELAY'].astype(int):
    if item < 0:
        arrival_delay.append('-' + str(datetime.timedelta(minutes = (-1 * item)))[:-3])
    else:
        arrival_delay.append(str(datetime.timedelta(minutes = item))[:-3])
        
for item in earlyarrivals['Total Delay'].astype(int):
    if item < 0:
        total_delay.append('-' + str(datetime.timedelta(minutes = (-1 * item)))[:-3])
    else:
        total_delay.append(str(datetime.timedelta(minutes = item))[:-3])
        
earlyarrivals['DEPARTURE_DELAY'] = departure_delay
earlyarrivals['ARRIVAL_DELAY'] = arrival_delay
earlyarrivals['TOTAL_DELAY'] = total_delay

'TAXI_OUT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'TAXI_IN'


taxi_out = []
scheduled_time = []
elapsed_time = []
air_time = []
taxi_in = []

for item in earlyarrivals['TAXI_OUT'].astype(int):
        taxi_out.append(str(datetime.timedelta(minutes = item))[:-3])

for item in earlyarrivals['SCHEDULED_TIME'].astype(int):
        scheduled_time.append(str(datetime.timedelta(minutes = item))[:-3])
        
for item in earlyarrivals['ELAPSED_TIME'].astype(int):
        elapsed_time.append(str(datetime.timedelta(minutes = item))[:-3])
        
for item in earlyarrivals['AIR_TIME'].astype(int):
        air_time.append(str(datetime.timedelta(minutes = item))[:-3])
        
for item in earlyarrivals['TAXI_IN'].astype(int):
        taxi_in.append(str(datetime.timedelta(minutes = item))[:-3])
        
earlyarrivals['TAXI_OUT'] = taxi_out
earlyarrivals['SCHEDULED_TIME'] = scheduled_time
earlyarrivals['ELAPSED_TIME'] = elapsed_time
earlyarrivals['AIR_TIME'] = air_time
earlyarrivals['TAXI_IN'] = taxi_in

C:\Users\sedra\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sedra\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sedra\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [12]:
diverted_df = update5[update5['DIVERTED']==1]
diverted_df.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,Total Delay,DATE
229537,1,AA,74,N436AA,DFW,HOU,920,927.0,7.0,17.0,...,247,1248.0,4.0,1026,1252.0,NaN,1,0,NaN,2015-02-16
230323,1,WN,1150,N7747C,BWI,MCO,1005,1022.0,17.0,9.0,...,787,1536.0,7.0,1240,1543.0,NaN,1,0,NaN,2015-02-16
230413,1,WN,947,N771SA,PHX,LIT,1010,1009.0,-1.0,26.0,...,1136,NaN,NaN,1350,NaN,NaN,1,0,NaN,2015-02-16
231311,1,AA,1672,N4YEAA,LAX,AUS,1105,504.0,1079.0,14.0,...,1242,NaN,NaN,1602,NaN,NaN,1,0,NaN,2015-02-16
231471,1,DL,1097,N397DA,LAX,RDU,1115,1114.0,-1.0,14.0,...,2239,NaN,NaN,1855,NaN,NaN,1,0,NaN,2015-02-16


In [13]:
diverted_df2 = diverted_df.drop(columns =['ARRIVAL_TIME','ARRIVAL_DELAY','TAXI_IN','WHEELS_ON','AIR_TIME','DEPARTURE_TIME','Total Delay','ELAPSED_TIME','WHEELS_OFF','TAXI_OUT','DEPARTURE_DELAY'])
diverted_df2
diverted_df3 = diverted_df2.dropna()
diverted_df3.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,CANCELLED,DATE
229537,1,AA,74,N436AA,DFW,HOU,920,66.0,247,1026,1,0,2015-02-16
230323,1,WN,1150,N7747C,BWI,MCO,1005,155.0,787,1240,1,0,2015-02-16
230413,1,WN,947,N771SA,PHX,LIT,1010,160.0,1136,1350,1,0,2015-02-16
231311,1,AA,1672,N4YEAA,LAX,AUS,1105,177.0,1242,1602,1,0,2015-02-16
231471,1,DL,1097,N397DA,LAX,RDU,1115,280.0,2239,1855,1,0,2015-02-16


### Tweet Data (Natalie)

MK NOTES
1. Columns to Drop: 'negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 'negativereason_gold', 'tweet_coord', 'tweet_location', 'user_timezone'
2. Convert Date Column to Only Include Date & Not Time
3. Convert Airline Name to Airline Code

In [14]:
#ADD CODE HERE

## Data Transformation

### Flight Data (Myles & Sedra)

MK NOTES
1. Airport Data - No Changes
2. Airline Data - No Changes
3. Flight Data - Use Group By & Filtering to Create Individual Data Frames for Delayed/Early/Cancelled/Diverted Flights for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [18]:
#Early Loops
early_group = earlyarrivals.groupby(['AIRLINE'])
for name, group in early_group:
    if name == 'UA':
        UA_early = pd.DataFrame(group)
        UA_early= UA_early.reset_index().drop(['index'], axis = 1)
    elif name == 'AA':
        AA_early = pd.DataFrame(group)
        AA_early = AA_early.reset_index().drop(['index'],axis = 1)
    elif name == 'DL':
        DL_early = pd.DataFrame(group)
        DL_early = DL_early.reset_index().drop(['index'],axis = 1)
    elif name == 'US':
        US_early = pd.DataFrame(group)
        US_early = US_early.reset_index().drop(['index'],axis = 1)
    elif name =='WN':
        WN_early = pd.DataFrame(group)
        WN_early = WN_early.reset_index().drop(['index'],axis = 1)
    elif name == 'VX':
        VX_early = pd.DataFrame(group)
        VX_early = VX_early.reset_index().drop(['index'],axis = 1)
print(UA_early)
print(AA_early)
print(DL_early)
print(US_early)
print(WN_early)

       DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER ORIGIN_AIRPORT  \
0                1      UA           1512      N33292            LAS   
1                1      UA           1200      N68805            SFO   
2                1      UA           1496      N17245            LAX   
3                1      UA           1204      N37273            SJU   
4                1      UA           1162      N37287            BQN   
5                1      UA            288      N424UA            EWR   
6                1      UA            260      N435UA            ORD   
7                1      UA            606      N807UA            MSY   
8                1      UA           1447      N27733            PHL   
9                1      UA           1573      N11206            SEA   
10               1      UA           1545      N17233            DFW   
11               1      UA            572      N842UA            AUS   
12               1      UA           1215      N69830           

In [19]:
#DIVERTED LOOPS
diverted_group = diverted_df3.groupby(['AIRLINE'])
for name, group in diverted_group:
    if name == 'UA':
        UA_diverted = pd.DataFrame(group)
        UA_diverted= UA_diverted.reset_index().drop(['index'], axis = 1)
    elif name == 'AA':
        AA_diverted = pd.DataFrame(group)
        AA_diverted = AA_diverted.reset_index().drop(['index'],axis = 1)
    elif name == 'DL':
        DL_diverted = pd.DataFrame(group)
        DL_diverted = DL_diverted.reset_index().drop(['index'],axis = 1)
    elif name == 'US':
        US_diverted = pd.DataFrame(group)
        US_diverted = US_diverted.reset_index().drop(['index'],axis = 1)
    elif name =='WN':
        WN_diverted = pd.DataFrame(group)
        WN_diverted = WN_diverted.reset_index().drop(['index'],axis = 1)
    elif name == 'VX':
        VX_diverted = pd.DataFrame(group)
        VX_diverted = VX_diverted.reset_index().drop(['index'],axis = 1)
print(UA_diverted)
print(AA_diverted)
print(DL_diverted)
print(US_diverted)
print(WN_diverted)

    DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER ORIGIN_AIRPORT  \
0             1      UA           1060      N14242            SFO   
1             1      UA           1188      N76514            SFO   
2             1      UA            536      N430UA            MIA   
3             1      UA           1261      N75425            IAH   
4             1      UA            464      N415UA            IAH   
5             2      UA            253      N772UA            IAH   
6             4      UA            687      N471UA            SFO   
7             5      UA           1142      N66831            MCO   
8             5      UA           1071      N76529            EWR   
9             6      UA           1109      N36472            EWR   
10            7      UA           1501      N14237            IAH   
11            2      UA            236      N473UA            LAS   

   DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  SCHEDULED_TIME  DISTANCE  \
0                  DCA       

### Tweet Data (Natalie)

MK NOTES
1. Use Group By & Filtering to Create Data Frames of Positive/Negative/Neutral Tweets for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [ ]:
#ADD CODE HERE

## Data Loading (Matt)

MK Notes
1. Create ERD for Completed Database
2. Create Database in PGAdmin
3. Use SQL Alchemy to Load Dataframes into Tables & Relate by Keys

In [ ]:
#ADD CODE HERE

# Project Analysis

## Data Extraction

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Transformation

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Loading (Matt)

ADD TEXT HERE